# Cloud comp: Ass 2

Idea:

* Calculate polarity score per tweet
* Append score to the end of tweet
* Return revised tweets in JSON

Notes on polarity score:
* Compound is [-1,1] where >0 is positive sentiment
* The others look at what percentage of tweet is neg, neutral or positive

The polarity score is stored in sentiment dictionary which is attached to the end of the tweet:
* sentiment: gives the sentiment score
* overall_sentiment: labels the score as positive, neutral or negative depending on compound value

Get CouchDB to handle the majority of processing. Still need tweets with suburban tags

## Import libraries

In [34]:
import re
import json
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer

nltk.download('vader_lexicon')

#Try with tiny Twitter. Small one takes about 20 mins!
twitter_file = 'smallTwitter.json'

outfile = 'revised_smallTwitter.json'

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Thomas\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


## 1. Use NLTK sentiment analyzer to calculate score

In [35]:
'''
Using NLTK sentiment analyzer
'''
tt = open(twitter_file,'r',encoding='utf-8').read()
twitter_data = json.loads(tt)


# Total number of tweets in file
number_of_tweets = len(twitter_data['rows']) 


sia = SentimentIntensityAnalyzer()

updated_tweets = []

for tweets in range(number_of_tweets):
    tweet = twitter_data["rows"][tweets]
    
    ##Process text file
    tweet_text = tweet["value"]["properties"]["text"].lower()
    
    sentiment_tweet_dict = {}

    sentiment_score = sia.polarity_scores(tweet_text)

    sentiment_score_compound = sentiment_score['compound']
    
    sentiment_tweet_dict['sentiment'] = sentiment_score

    if sentiment_score_compound == 0:
        overall_sentiment = 'neutral'
    elif sentiment_score_compound > 0:
        overall_sentiment = 'positive'
    else:
        overall_sentiment = 'negative'
    
    sentiment_tweet_dict['overall_sentiment'] = overall_sentiment
    
    tweet.update(sentiment_tweet_dict)
    
    updated_tweets.append(tweet)

## 2. Writes tweets into another JSON file

In [36]:
revised_tweets = {}

revised_tweets['rows'] = updated_tweets

with open(outfile,'w') as jsonFile:
    json.dump(revised_tweets, jsonFile)